In [1]:
import math,csv, random, collections
import numpy as np
import pickle
import seaborn as sns
import matplotlib.patches as mpatches
import random, string

import imp, sys
sys.path.insert(0, '../')
import utils, algorithms
import h5py, imp

import matplotlib.pyplot as plt
from IPython.display import display, HTML
from multiprocessing import Pool
from multiprocessing import Process, Queue
imp.reload(utils)
imp.reload(algorithms)
        
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
sns.set_style('dark')
color_p = sns.color_palette("hls", 10*2)

In [2]:
def BP(M, n_steps, filename):
    n = M.shape[0]
    BP_matrix = np.zeros((2, n, n)) #BP_matrix[i] will store the state of BP in step i
    BP_matrix[0] = M.copy() #First state of BP_matrix is the data matrix itself
    accuracy = np.zeros(n_steps) #Stores accuracy after each step
    
    #True answer
    true_answer = np.zeros((n,2))
    true_answer[:,0] = np.roll(np.arange(n),1)
    true_answer[:,1] = np.roll(np.arange(n),-1)
    true_answer = true_answer.astype(int)

    for l in range(1, n_steps):
        for i in range(n):
            for j in range(n):
                row = BP_matrix[(l-1)%2][:,i].copy()
                row[j] = np.inf
                BP_matrix[l%2][i][j] = M[i][j] - np.partition(row, 1)[1]
                
        BP_matrix[l%2][np.diag_indices_from(BP_matrix[l%2])] = np.inf
        
        M_final = BP_matrix[l%2]
        M_final[np.diag_indices_from(M_final)] = np.inf
        answer = M_final.argsort(axis=1)[:,:2]
        answer.sort(axis=1)
        answer[0] = answer[0][::-1]
        answer[-1] = answer[-1][::-1]
        accuracy[l] = np.mean(answer==true_answer)
        if accuracy[l] == 1:
            accuracy[l:] = accuracy[l]
            break
        if l > 75 and abs(np.mean(accuracy[l-25:l]) - np.mean(accuracy[l-50:l-25])) < 0.001: 
            accuracy[l:] = np.mean(accuracy[l-20:l])
            break
    print "BP", l , "Steps"
    output = ','.join(np.array(accuracy[1:]).astype(str))
    output = str(n)+","+str(mu)+","+str(l)+","+output
    fd = open(filename,'a')
    fd.write(output+"\n")
    fd.close()




def LP(data, file):
    n = data.shape[0]
    model , t      = algorithms.Solve_LP(data)
    answer         = np.array(model.X).reshape(n,n)
    correct_answer = algorithms.correct_data_answer(data,k=1)
           
    error = (np.where(correct_answer-answer!=0)[0].shape[0]+0.0)/(2*n)
    fp = (np.where(correct_answer-answer==-1)[0].shape[0]+0.0)/(2*n)
    point5 = (np.where(answer==0.5)[0].shape[0]+0.0)/(2*n)
    if file != None:
        fd = open(file,'a')
        fd.write(', '.join([str(n), str(mu), str(error), str(fp), str(point5),"\n"]))
        fd.close()

In [ ]:
n = 1000
it_limit = np.sqrt(4*np.log(n))
mg_limit = np.sqrt(6*np.log(n))
sg_limit = np.sqrt(8*np.log(n))
mu_array1 = np.linspace(it_limit*.8,it_limit*1.2,8)
mu_array2 = np.linspace(it_limit*.95,it_limit*1.05,8)
mu_array = np.concatenate([mu_array1, mu_array2])
mu_array[np.abs(mu_array-it_limit).argmin()] = it_limit
mu_array[np.abs(mu_array-sg_limit).argmin()] = sg_limit

mu_array.sort()
# M = M.astype(int)
print "it = ", it_limit
print "mg = ", mg_limit
print "sg = ", sg_limit
no_exp = 50

filenameBP = 'data/BP_'+str(n)+'.csv'   
filenameLP = 'data/LP_'+str(n)+'.csv'   

for mu in mu_array:
    print "\n", mu,
    no_process = 32
    pool = Pool(processes=no_process)  
    processes = []
    for exp in range(no_exp):
        print " ", exp+1,
        M = algorithms.Gaussian_matrix(mu=mu, n =n)
        
        #BP
        M[np.diag_indices_from(M)]= -np.inf
        args = (-M, n*10, filenameBP)
        p = Process(target = BP, args = args)
        p.start()
        processes.append(p)
        
        #LP
        M[np.diag_indices_from(M)]= 0
        args = (M, filenameLP)
        p = Process(target = LP, args = args)
        p.start()
        processes.append(p)

    print  "\nJoining Process"
    for j, p in enumerate(processes):
        print j, "\t",
        p.join()

#         accuracy_array[i][exp], _, _ = BP(-M,n)
#         break
#         print "\nSG, MG, LP", accuracy_array[i][exp][1], accuracy_array[i][exp][2], accuracy_array[i][exp][-1]

it =  5.25652176976
mg =  6.43789807887
sg =  7.4338443777

4.20521741581   1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50 
Joining Process
0 	 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
1 	2 	 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
 BP 76 Steps
3 	4 	 BP 76 Steps
5 	6 	7 	8 	 BP 76 Steps
 BP 76 Steps
9 	10 	 BP 77 Steps
 BP 77 Steps
 BP 80 Steps
 BP 80 Steps
 BP 81 Steps
 BP 82 Steps
 BP 83 Steps
 BP 84 Steps
 BP 87 Steps
 BP 87 Steps
 BP 90 Steps
 BP 92 Steps
 BP 91 Steps
 BP 93 Steps
 BP 97 Steps
 BP 131 Steps
 BP 131 Steps
 BP 140 Steps
 BP 172 Steps
 BP 175 Steps
11 	1

In [ ]:
plt.figure(figsize=(22,13))
xlim = 20
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 25,
        }

x = np.arange(0,n-1)
for i, mu in enumerate(mu_array):
    y = accuracy_array[i]
    y = y.mean(axis=0)
    
    greedy_val = float("{0:.3f}".format((1.000-y[0]))) 
    if greedy_val > 0.01:
        plt.annotate('SG='+str(greedy_val), xy=(0, 1.0001-y[0]), xytext=(.5, (1.0001-y[0])*1.1),
                arrowprops=dict(facecolor=color_p[i%5], shrink=0.01, width=1),
                )
    
    merge_greedy_val = float("{0:.3f}".format((1.000-y[1]))) 
#     if merge_greedy_val > 0.01:

#         plt.annotate('MG='+str(merge_greedy_val), xy=(1, 1.0001-y[1]),  xytext=(1.7, (1.0001-y[0])*1.1),
#                 arrowprops=dict(facecolor=color_p[i%5], shrink=0.01, width=1),
#                 )
    
    BP_val = float("{0:.4f}".format((1.000-y[-1]))) 
    plt.annotate('BP='+str(BP_val), xy=(xlim, 1.0001-y[xlim]),  xytext=(xlim-2, (1.0001-y[xlim])*1.1),
            arrowprops=dict(facecolor=color_p[i%5], shrink=0.01, width=1),
            )
    
    plt.semilogy(1.0001-y, c=color_p[i%5], lw=3)
    
    
    x_text = min(xlim/1.5, y.argmax())*1.1
    font_mu = {'family': 'serif',
        'color':  color_p[i%5],
        'weight': 'normal',
        'size': 15,
        }
    if i < 5:
        plt.text(x_text, (1.0001-y[x_text])*1.1, r'$\mu=$ '+str(float("{0:.4f}".format(mu))), fontdict=font_mu)

    if abs(mu - it_limit) < 0.00001:
        plt.text(4.2, (1.0001-y[4]), r'$\mu=$ '+str(float("{0:.4f}".format(mu))), fontdict=font_mu)

        plt.fill_between(x, 1.0001-y, np.ones_like(y) , color=color_p[4], alpha=0.2, label="<IT limit")
        plt.text(8, .05, r'$\mu<$ IT limit', fontdict=font)
        plt.fill_between(x, np.ones_like(y)*0.0001, 1.0001-y, color=color_p[0], alpha=0.2, label=">IT limit")
        plt.text(2, .0005, r'$\mu>$ IT limit', fontdict=font)
    
    
plt.xlabel("Steps", fontsize=20)
plt.ylabel("Error")
plt.title("BP algorithm with $\mu_{IT}$ = "+str(float("{0:.4f}".format(it_limit))) , fontsize=20)
plt.ylim((0,1))
plt.xlim((0,20))
plt.show()

In [2]:
true_answer = np.zeros((n,2))
true_answer[:,0] = np.roll(np.arange(n),1)
true_answer[:,1] = np.roll(np.arange(n),-1)
true_answer = true_answer.astype(int)
acc = np.zeros(n)
for l in range(n-1):
    M_final = BP_matrix[l]
    M_final[np.diag_indices_from(M_final)] = np.inf
    answer = M_final.argsort(axis=1)[:,:2]
    answer.sort(axis=1)
    answer[0] = answer[0][::-1]
    answer[-1] = answer[-1][::-1]
    acc[l] = np.mean(answer==true_answer)
    print l, acc[l], " ",

#     print l, np.where(BP_map[l]!=BP_map[i+1])[0].shape[0], np.prod(BP_map[i].shape)

NameError: name 'np' is not defined

In [3]:
plt.figure(figsize=(15,15))
plt.plot(acc[:-1])
plt.show()

NameError: name 'plt' is not defined

In [63]:
np.set_printoptions(precision=2)
for i in range(0,3):
    print i
    print BP_matrix[i]
    print answers[i], collections.Counter(answers[i].flatten())

0
[[  inf -0.7   2.78  2.9   4.1   2.75  3.23  3.32  3.11  0.07]
 [-0.7    inf -1.05  3.5   4.21  2.47  4.15  2.13  3.1   1.12]
 [ 2.78 -1.05   inf  0.35  2.94  2.68  3.87  4.31  1.17  1.8 ]
 [ 2.9   3.5   0.35   inf  0.7   1.14  4.38  2.6   2.9   2.02]
 [ 4.1   4.21  2.94  0.7    inf  0.59  3.15  3.85  1.34  3.99]
 [ 2.75  2.47  2.68  1.14  0.59   inf  1.05  3.82  2.61  2.81]
 [ 3.23  4.15  3.87  4.38  3.15  1.05   inf -0.81  6.52  3.14]
 [ 3.32  2.13  4.31  2.6   3.85  3.82 -0.81   inf  1.57  3.37]
 [ 3.11  3.1   1.17  2.9   1.34  2.61  6.52  1.57   inf -2.48]
 [ 0.07  1.12  1.8   2.02  3.99  2.81  3.14  3.37 -2.48   inf]]
[[9 1]
 [0 2]
 [1 3]
 [2 4]
 [3 5]
 [4 6]
 [5 7]
 [6 8]
 [2 9]
 [8 0]] Counter({2: 3, 0: 2, 1: 2, 3: 2, 4: 2, 5: 2, 6: 2, 8: 2, 9: 2, 7: 1})
1
[[  inf -3.46  2.71  2.83  4.03  2.69  3.16  3.26  3.04 -2.69]
 [-1.83   inf -2.18  4.2   4.91  3.18  4.85  2.84  3.81  1.83]
 [ 2.43 -2.22   inf -0.82  2.59  2.33  3.52  3.96  0.82  1.45]
 [ 2.2   2.79 -0.79   inf -0.44  0.

In [64]:
k = 2
for i in range(0,6):
    print i
    print -M[k], np.array(sorted(BP_matrix[i][k]))[:3]
    print BP_tmp[i+1][k]
    print BP_matrix[i+1][k]
    print answers[i][k]

0
[ 2.78 -1.05   inf  0.35  2.94  2.68  3.87  4.31  1.17  1.8 ] [-1.05  0.35  1.17]
[ 0.35  1.17  0.35  1.17  0.35  0.35  0.35  0.35  0.35  0.35]
[ 2.43 -2.22   inf -0.82  2.59  2.33  3.52  3.96  0.82  1.45]
[1 3]
1
[ 2.78 -1.05   inf  0.35  2.94  2.68  3.87  4.31  1.17  1.8 ] [-2.22 -0.82  0.82]
[-0.79 -0.17 -0.79 -0.17 -0.79 -0.79 -0.79 -0.79 -0.79 -0.79]
[ 3.57 -0.88   inf  0.53  3.73  3.47  4.66  5.1   1.96  2.59]
[1 3]
2
[ 2.78 -1.05   inf  0.35  2.94  2.68  3.87  4.31  1.17  1.8 ] [-0.88  0.53  1.96]
[ 0.26  1.74  0.26  1.74  0.26  0.26  0.26  0.26  0.26  0.26]
[ 2.52 -2.79   inf -1.38  2.69  2.43  3.62  4.06  0.92  1.54]
[1 3]
3
[ 2.78 -1.05   inf  0.35  2.94  2.68  3.87  4.31  1.17  1.8 ] [-2.79 -1.38  0.92]
[-1.54  1.5  -1.54  1.5  -1.54 -1.54 -1.54 -1.54 -1.54 -1.54]
[ 4.32 -2.55   inf -1.14  4.49  4.23  5.42  5.85  2.71  3.34]
[1 3]
4
[ 2.78 -1.05   inf  0.35  2.94  2.68  3.87  4.31  1.17  1.8 ] [-2.55 -1.14  2.71]
[-0.37  2.84 -0.37  2.84 -0.37 -0.37 -0.37 -0.37 -0.37 -0.37

In [42]:
for i in range(len(answers)-1):
    print np.where(answers[i]!=answers[i+1])

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [9]:
def BP(M, n_steps):
    n = M.shape[0]
    BP_matrix = np.zeros((n_steps, n, n)) #BP_matrix[i] will store the state of BP in step i
    BP_matrix[0] = M.copy() #First state of BP_matrix is the data matrix itself
    accuracy = np.zeros(n_steps) #Stores accuracy after each step
    
    #True answer
    true_answer = np.zeros((n,2))
    true_answer[:,0] = np.roll(np.arange(n),1)
    true_answer[:,1] = np.roll(np.arange(n),-1)
    true_answer = true_answer.astype(int)

    for l in range(1, n_steps):
        for i in range(n):
            for j in range(n):
                row = BP_matrix[l-1][:,i].copy()
                row[j] = np.inf
                BP_matrix[l][i][j] = M[i][j] - np.partition(row, 1)[1]
        BP_matrix[l][np.diag_indices_from(BP_matrix[l])] = np.inf
        

        if accuracy[l] == 1:
            accuracy[l:] = accuracy[l]
            break
        if l > 10 and np.mean(accuracy[l-5:l]==accuracy[l-10:l-5]) == 1:
            accuracy[l:] = accuracy[l]
            break
    M_final = BP_matrix[l]
    M_final[np.diag_indices_from(M_final)] = np.inf
    topTwo = M_final.argsort(axis=1)[:,:2]
    answer = np.zeros((n,n))
    for i in range(n):
        answer[i][topTwo[i]] =1
    return answer

M = algorithms.Gaussian_matrix(mu=10, n =50)
answer = BP(-M,n)


In [13]:
answer

correct_answer = algorithms.correct_data_answer(M,k=1)

error = (np.where(correct_answer-answer!=0)[0].shape[0]+0.0)/(2*n)
fp = (np.where(correct_answer-answer==-1)[0].shape[0]+0.0)/(2*n)
print "LP", 1, error, fp

 LP 1 0.0 0.0
